# Week 2 - eCommerce Assistant for products price from dictionary

An eCommerce assitant that can get a product price

Gradio for chat box

In [ ]:
# !ollama pull llama3.2

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
system_message = "You are a helpful assistant for an online store called CommerceAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [ ]:
# Let's start by making a useful function

items_prices = {f"item{i}": f"{i*100}" for i in range(1,6)}

items_prices = {"printer": "$500", "paper": "$10", "mini printer": "$50", "label printer": "$60", "sticker-paper": "$5"}

def get_item_price(product):
    print(f"Tool get_item_price called for {product}")
    item = product.lower()
    return items_prices.get(item, "Unknown")

In [ ]:
items_prices

In [ ]:
get_item_price("mini printer")

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_item_price",
    "description": "Get the price of an item in the store. \
                    Call this whenever you need to know the store item price , \
                    for example when a customer asks 'How much is a mini printer' ",
    "parameters": {
        "type": "object",
        "properties": {
            "product": {
                "type": "string",
                "description": "The item that the customer wants to buy"
            },
        },
        "required": ["product"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, item = handle_tool_call(message)
        print('response', response, 'item', item)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    item = arguments.get('product') 
    print('product', item)
    price = get_item_price(item)
    response = {
        "role": "tool",
        "content": json.dumps({"item": item,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, item
    

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()